In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np

import sklearn

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5 
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix'

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory = False)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth = 5)):
  x = df[feats].values
  y = df['prices_amountmin'].values
  
  scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [0]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
model = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
df.features.head().values

In [0]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

In [0]:
literal_eval(str_dict)[0]['value']

['Men']

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  return output_dict
df['features_parsed'] = df['features'].map(parse_features)

In [0]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)


476

In [0]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan) 

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_fit to', 'feat_chest size', 'feat_inv', 'feat_manufacturer',
       'feat_power type', 'feat_shape', 'feat_item color',
       'feat_lens socket width', 'feat_sock size',
       'feat_country of origin - assembly'],
      dtype='object', length=526)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False ==df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100

In [0]:
{k:v for k,v in keys_stat.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer_part_number_cat'] = df['feat_manufacturer_part_number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat'] = df[get_name_feat(key)].factorize()[0]
  get_name_feat(key)

In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [0]:
feats = ['']

In [160]:
model = RandomForestRegressor(max_depth = 5, n_estimators = 100)
run_model(feats ,model)




(-57.083407533545106, 4.307504961077651)

In [171]:
feats_cat = [x for x in df.columns if 'cat' in x]
feats_cat

['categories',
 'brand_cat',
 'feat_multi pack indicator',
 'feat_location - country',
 'feat_certifications and listings',
 'feat_fabrication',
 'feat_location - city/state',
 'feat_recommended location',
 'feat_clothing category',
 'feat_shoe category',
 'feat_catalog',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer_part_number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_item weight_cat',
 'feat_style code_cat',
 'feat_frame material:_cat',
 'feat_case material_cat',
 'feat_resizable_cat',
 'feat_diameter_cat',
 'feat_jewelry setting_cat',
 'feat_purpose_cat',
 'feat_weight_cat',
 'feat_frame depth_cat',
 'feat_polarised_cat',
 'feat_fabric care_cat',
 'feat_transactionid_cat',
 'feat_to fit_cat',
 'feat_polarized?_cat',
 'feat_weather resistant_cat',
 'feat_is waterproof_cat',
 'feat_work shoes_cat',
 'feat_has adaptive lenses_cat',
 'feat_band width_cat',
 'feat_labor_cat',
 'feat_display_cat',
 'feat_condition_cat',
 'fea

In [0]:
feats = ['brand_cat','feat_brand_cat','feat_shoe category_cat','feat_movement_cat','feat_age_cat','feat_gender_cat','feat_material_cat','feat_style_cat','feat_sport_cat']
 
#feats += feats_cat
#feats = list(set(feats))

In [0]:
model = RandomForestRegressor(max_depth = 5, n_estimators = 100)
result = run_model(feats ,model)


In [207]:
x = df[feats].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(x,y)

print(result)

perm = PermutationImportance(m,random_state=1).fit(x,y);
eli5.show_weights(perm, feature_names=feats)

(-57.43392470148412, 4.3348687864293245)


Weight,Feature
0.2592 ± 0.0129,brand_cat
0.1044 ± 0.0063,feat_material_cat
0.0226 ± 0.0019,feat_gender_cat
0.0153 ± 0.0013,feat_brand_cat
0.0131 ± 0.0006,feat_shoe category_cat
0.0109 ± 0.0008,feat_age_cat
0.0108 ± 0.0008,feat_movement_cat
0.0048 ± 0.0004,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [208]:
df[df['brand']=='nike'].features_parsed.sample(5).values

array([{'sport': 'soccer', 'condition': 'new with box', 'type': 'cleats'},
       {'sport': 'football, lacrosse', 'type': 'cleats', 'condition': 'new without box'},
       {'brand': 'nike', 'manufacturer_part_number': '835570-701', 'fabric content': '100% woven', 'color': 'volt/black-white'},
       {'season': 'all-season', 'material': 'leather', 'gender': 'men', 'shoe size': '7.5', 'color': 'brown', 'manufacturer part number': '555317 240', 'brand': 'nike', 'age group': 'adult', 'size': '6.5', 'model': '555317 240'},
       {'style': 'shirts & tops', 'condition': 'new with tags'}],
      dtype=object)